In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/SkinCancer/input/train'
valid_path = '/content/drive/MyDrive/SkinCancer/input/test'

# add preprocessing layer to the front of VGG
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in resnet.layers:
  layer.trainable = False
  
data_aug = False
  
  # useful for getting number of classes
folders = glob('/content/drive/MyDrive/SkinCancer/input/train/*')
  

# our layers - you can add more if you want
x = Flatten()(resnet.output)
x = Dense(128, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/SkinCancer/input/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/SkinCancer/input/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

'''r=model.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 5,
                         validation_data = test_set,
                         nb_val_samples = 2000)'''
if data_aug:
  # fit the model
  print ("Training with data augmentation")
  r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=5,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
  )
else:
  print ("Training with early stop")
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto', restore_best_weights=True)
  # fit model
  history = model.fit(training_set, epochs=200, batch_size=20, validation_data=test_set, verbose=1, callbacks=[monitor])



In [ ]:
# loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.savefig('/content/drive/MyDrive/Models_100/ResNet50/LossVal_ResNet50_loss128.png')
plt.legend()
plt.show()


# accuracies
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.savefig('/content/drive/MyDrive/Models_100/ResNet50/AccVal_ResNet50_loss128.png')
plt.legend()
plt.show()


import tensorflow as tf

from keras.models import load_model

model.save('/content/drive/MyDrive/Models_100/ResNet50/Resnet50_128.h5')
